In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
from torchvision.datasets import ImageFolder
import os
import shutil
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
train_folder = 'train_aug'
val_folder = "validation_dataset"

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train = ImageFolder(train_folder, transform=transform)
val = ImageFolder(val_folder, transform=transform)

train_loader = DataLoader(train, batch_size=512, shuffle=True)
val_loader = DataLoader(val, batch_size=512, shuffle=False)







In [3]:
model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
# add at least two more layers with a softmax activation function (your output layer should have 30 neurons and the layer before the output layer should have 8 neurons).

model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 8),
    nn.ReLU(),
    nn.Linear(8, 30),
    nn.Softmax(dim=1)
)
               

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/Users/himanshi/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/himanshi/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
for epoch in range(2):
    losses = []

    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print(f'Epoch - {epoch + 1}, Loss - {sum(losses)/len(losses)}')


for param in model.parameters():
    param.requires_grad = True

for epoch in range(2):
    losses = []

    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print(f'Epoch - {epoch + 1}, Loss - {sum(losses)/len(losses)}')


# save weights 
torch.save(model.state_dict(), 'model_vgg16.pth')


model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy of the network on the validation images: {100 * correct / total}%')






100%|██████████| 8/8 [01:24<00:00, 10.59s/it]


Epoch - 1, Loss - 3.3902179300785065


100%|██████████| 8/8 [01:23<00:00, 10.43s/it]


Epoch - 2, Loss - 3.3553682267665863


100%|██████████| 8/8 [05:55<00:00, 44.45s/it]


Epoch - 1, Loss - 3.3010585010051727


100%|██████████| 8/8 [05:54<00:00, 44.31s/it]


Epoch - 2, Loss - 3.265322983264923
Accuracy of the network on the validation images: 17.5%


In [5]:
model = torchvision.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 8),
    nn.ReLU(),
    nn.Linear(8, 30),
    nn.Softmax(dim=1)
)


model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/Users/himanshi/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
for epoch in range(10):
    losses = []

    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print(f'Epoch - {epoch + 1}, Loss - {sum(losses)/len(losses)}')


for param in model.parameters():
    param.requires_grad = True

for epoch in range(10):
    losses = []

    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print(f'Epoch - {epoch + 1}, Loss - {sum(losses)/len(losses)}')


# save weights
torch.save(model.state_dict(), 'model_resnet50.pth')


model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy of the network on the validation images: {100 * correct / total}%')




100%|██████████| 8/8 [02:10<00:00, 16.32s/it]


Epoch - 1, Loss - 3.4003223180770874


100%|██████████| 8/8 [02:07<00:00, 15.92s/it]


Epoch - 2, Loss - 3.399769574403763


100%|██████████| 8/8 [02:09<00:00, 16.21s/it]


...

100%|██████████| 8/8 [02:11<00:00, 16.38s/it]


Epoch - 10, Loss - 3.3781368136405945


100%|██████████| 8/8 [04:28<00:00, 33.52s/it]


Epoch - 1, Loss - 3.3247534334659576


 62%|██████▎   | 5/8 [03:12<01:55, 38.48s/it]